In [1]:
# Install required packages

!pip install --upgrade xgboost

In [2]:
# Import libraries and global settings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from itertools import product
from sklearn.metrics import mean_squared_error

pd.set_option("display.max_columns", 100)
RANDOM_STATE = 42


In [3]:
# Load all dataset CSV files

sales = pd.read_csv("sales_train.csv")
test = pd.read_csv("test.csv")
items = pd.read_csv("items.csv")
sample = pd.read_csv("sample_submission.csv")
shops = pd.read_csv("shops.csv")
item_cats = pd.read_csv("item_categories.csv")

sales.head()


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
# Clean sales data: clip outliers and parse dates

sales["item_cnt_day"] = sales["item_cnt_day"].clip(0, 20)
sales["item_price"] = sales["item_price"].clip(0, sales["item_price"].quantile(0.99))

sales["date"] = pd.to_datetime(sales["date"], format="%d.%m.%Y")

print(sales.shape)
sales.head()


(2935849, 6)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,0.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0


In [5]:
# Create monthly item_cnt_month values

monthly = (
    sales
    .groupby(["date_block_num", "shop_id", "item_id"], as_index=False)["item_cnt_day"]
    .sum()
    .rename(columns={"item_cnt_day": "item_cnt_month"})
)

monthly["item_cnt_month"] = monthly["item_cnt_month"].clip(0, 20)
monthly.head()


,date_block_num,shop_id,item_id,item_cnt_month
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0


In [6]:
# Create full grid of date_block_num, shop_id, item_id for all months & active combos

matrix = []

for block_num in range(34):  # 0-33 months in training
    block_data = sales[sales["date_block_num"] == block_num]
    shops_in_block = block_data["shop_id"].unique()
    items_in_block = block_data["item_id"].unique()
    matrix.append(
        np.array(list(product([block_num], shops_in_block, items_in_block)))
    )

matrix = pd.DataFrame(
    np.vstack(matrix),
    columns=["date_block_num", "shop_id", "item_id"]
).astype({"date_block_num": np.int8, "shop_id": np.int16, "item_id": np.int16})

matrix = matrix.sort_values(["date_block_num", "shop_id", "item_id"]).reset_index(drop=True)

# Merge monthly sales
matrix = matrix.merge(
    monthly,
    on=["date_block_num", "shop_id", "item_id"],
    how="left"
)
matrix["item_cnt_month"] = matrix["item_cnt_month"].fillna(0).clip(0, 20)

matrix.head()


,date_block_num,shop_id,item_id,item_cnt_month
0,0,0,19,0.0
1,0,0,27,0.0
2,0,0,28,0.0
3,0,0,29,0.0
4,0,0,32,6.0


In [7]:
# Test corresponds to the "next" month: date_block_num = 34

test_mod = test.copy()
test_mod["date_block_num"] = 34
test_mod["item_cnt_month"] = np.nan  # unknown target

test_mod = test_mod[["date_block_num", "shop_id", "item_id", "item_cnt_month"]]

# Append to matrix
matrix = pd.concat([matrix, test_mod], ignore_index=True, sort=False)

matrix.tail()


,date_block_num,shop_id,item_id,item_cnt_month
11128045,34,45,18454,NaN
11128046,34,45,16188,NaN
11128047,34,45,15757,NaN
11128048,34,45,19648,NaN
11128049,34,45,969,NaN


In [8]:
# Merge items.csv (gives item_category_id + item_name)

matrix = matrix.merge(
    items[["item_id", "item_category_id", "item_name"]],
    on="item_id",
    how="left"
)

# Merge item_categories.csv (adds category name)
matrix = matrix.merge(
    item_cats[["item_category_id", "item_category_name"]],
    on="item_category_id",
    how="left"
)

# Merge shops.csv
matrix = matrix.merge(
    shops[["shop_id", "shop_name"]],
    on="shop_id",
    how="left"
)


In [9]:
# Function: add lag features to the dataset

def add_lag_feature(df, lags, col_name):
    tmp = df[["date_block_num", "shop_id", "item_id", col_name]]
    for lag in lags:
        shifted = tmp.copy()
        shifted["date_block_num"] += lag
        shifted = shifted.rename(columns={col_name: f"{col_name}_lag_{lag}"})
        df = df.merge(
            shifted,
            on=["date_block_num", "shop_id", "item_id"],
            how="left"
        )
    return df


In [10]:
# Add 1–3 month lag features for item_cnt_month

matrix = add_lag_feature(matrix, [1, 2, 3], "item_cnt_month")

# Fill NaN lag values with 0 (no past info)
lag_cols = [c for c in matrix.columns if "lag_" in c]
matrix[lag_cols] = matrix[lag_cols].fillna(0)

matrix.head()


,date_block_num,shop_id,item_id,item_cnt_month,item_category_id,item_name,item_category_name,shop_name,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3
0,0,0,19,0.0,40,/ЗОЛОТАЯ КОЛЛЕКЦИЯ м/ф-72,Кино - DVD,"!Якутск Орджоникидзе, 56 фран",0.0,0.0,0.0
1,0,0,27,0.0,19,"007 Legends [PS3, русская версия]",Игры - PS3,"!Якутск Орджоникидзе, 56 фран",0.0,0.0,0.0
2,0,0,28,0.0,30,"007 Legends [PС, Jewel, русская версия]",Игры PC - Стандартные издания,"!Якутск Орджоникидзе, 56 фран",0.0,0.0,0.0
3,0,0,29,0.0,23,"007 Legends [Xbox 360, русская версия]",Игры - XBOX 360,"!Якутск Орджоникидзе, 56 фран",0.0,0.0,0.0
4,0,0,32,6.0,40,1+1,Кино - DVD,"!Якутск Орджоникидзе, 56 фран",0.0,0.0,0.0


In [11]:
# Add item-level and shop-level average sales lag features

group = (
    matrix[matrix["date_block_num"] < 34]  # don't use pure test rows
    .groupby(["date_block_num", "item_id"], as_index=False)["item_cnt_month"]
    .mean()
    .rename(columns={"item_cnt_month": "date_item_avg_cnt"})
)

matrix = matrix.merge(group, on=["date_block_num", "item_id"], how="left")
matrix = add_lag_feature(matrix, [1], "date_item_avg_cnt")
matrix.drop(columns=["date_item_avg_cnt"], inplace=True)

# Average monthly sales per shop
group = (
    matrix[matrix["date_block_num"] < 34]
    .groupby(["date_block_num", "shop_id"], as_index=False)["item_cnt_month"]
    .mean()
    .rename(columns={"item_cnt_month": "date_shop_avg_cnt"})
)

matrix = matrix.merge(group, on=["date_block_num", "shop_id"], how="left")
matrix = add_lag_feature(matrix, [1], "date_shop_avg_cnt")
matrix.drop(columns=["date_shop_avg_cnt"], inplace=True)

# Fill any remaining NaNs from aggregates
agg_lag_cols = [c for c in matrix.columns if ("avg_cnt_lag" in c)]
matrix[agg_lag_cols] = matrix[agg_lag_cols].fillna(0)

matrix.head()


,date_block_num,shop_id,item_id,item_cnt_month,item_category_id,item_name,item_category_name,shop_name,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,date_item_avg_cnt_lag_1,date_shop_avg_cnt_lag_1
0,0,0,19,0.0,40,/ЗОЛОТАЯ КОЛЛЕКЦИЯ м/ф-72,Кино - DVD,"!Якутск Орджоникидзе, 56 фран",0.0,0.0,0.0,0.0,0.0
1,0,0,27,0.0,19,"007 Legends [PS3, русская версия]",Игры - PS3,"!Якутск Орджоникидзе, 56 фран",0.0,0.0,0.0,0.0,0.0
2,0,0,28,0.0,30,"007 Legends [PС, Jewel, русская версия]",Игры PC - Стандартные издания,"!Якутск Орджоникидзе, 56 фран",0.0,0.0,0.0,0.0,0.0
3,0,0,29,0.0,23,"007 Legends [Xbox 360, русская версия]",Игры - XBOX 360,"!Якутск Орджоникидзе, 56 фран",0.0,0.0,0.0,0.0,0.0
4,0,0,32,6.0,40,1+1,Кино - DVD,"!Якутск Орджоникидзе, 56 фран",0.0,0.0,0.0,0.0,0.0


In [12]:
# Add month column and split into train/validation/test

matrix["month"] = matrix["date_block_num"] % 12

# Drop very early months where lags are not fully defined
matrix = matrix[matrix["date_block_num"] >= 3].reset_index(drop=True)

# Separate target and features
target_col = "item_cnt_month"

# Mask for train / validation / test
train_mask = matrix["date_block_num"] < 33
valid_mask = matrix["date_block_num"] == 33
test_mask  = matrix["date_block_num"] == 34

features = [
    c for c in matrix.columns
    if c not in [target_col, "ID"]  # ID only exists for some test rows
]

X_train = matrix.loc[train_mask, features]
y_train = matrix.loc[train_mask, target_col]

X_valid = matrix.loc[valid_mask, features]
y_valid = matrix.loc[valid_mask, target_col]

X_test = matrix.loc[test_mask, features]

X_train.shape, X_valid.shape, X_test.shape


((9552883, 13), (238172, 13), (214200, 13))

In [13]:
# Install LightGBM

!pip install lightgbm

In [14]:
# Import LGBMRegressor

from lightgbm import LGBMRegressor

In [15]:
# Define LightGBM model parameters

model = LGBMRegressor(
    max_depth=8,
    n_estimators=500,
    learning_rate=0.03,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)


In [16]:
# Drop text columns that LightGBM cannot use

X_train = X_train.drop(columns=['item_name', 'item_category_name', 'shop_name'], errors='ignore')
X_valid = X_valid.drop(columns=['item_name', 'item_category_name', 'shop_name'], errors='ignore')
X_test  = X_test.drop(columns=['item_name', 'item_category_name', 'shop_name'], errors='ignore')


In [17]:
# Train the initial LightGBM model

model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.123345 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1007
[LightGBM] [Info] Number of data points in the train set: 9552883, number of used features: 10
[LightGBM] [Info] Start training from score 0.294607


LGBMRegressor(colsample_bytree=0.8, learning_rate=0.03, max_depth=8,
              n_estimators=500, random_state=42, subsample=0.8)

In [ ]:
# Predict on train & validation sets

y_pred_valid = model.predict(X_valid)
y_pred_train = model.predict(X_train)

In [ ]:
# Compute RMSE for train and validation sets

from sklearn.metrics import mean_squared_error

rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
rmse_valid = np.sqrt(mean_squared_error(y_valid, y_pred_valid))


print(rmse_train, rmse_valid)
print("train", rmse_train)
print("valid", rmse_valid)


In [ ]:
# Plot top 20 feature importances

importances = model.feature_importances_
feat_names = np.array(features)

idx = np.argsort(importances)[::-1][:20]

plt.figure(figsize=(8, 6))
plt.barh(range(len(idx)), importances[idx])
plt.yticks(range(len(idx)), feat_names[idx])
plt.xlabel("Importance")
plt.title("Top 20 Feature Importances")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()



In [ ]:
# Create full training data (all months < 34)

train_full_mask = matrix["date_block_num"] < 34

# Full feature matrix for final model
X_full = matrix.loc[train_full_mask, features].drop(
    columns=['item_name', 'item_category_name', 'shop_name'],
    errors='ignore'
)

# Full target vector
y_full = matrix.loc[train_full_mask, target_col]


In [ ]:
# Drop metadata text columns from X_full

drop_cols = ['item_name', 'item_category_name', 'shop_name']
X_full = X_full.drop(columns=[c for c in drop_cols if c in X_full.columns])

In [ ]:
# Train the final LightGBM model

final_model = LGBMRegressor(
    n_estimators=500,
    learning_rate=0.03,
    num_leaves=128,
    max_depth=-1,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.5,
    reg_lambda=0.5,
    min_child_samples=50,
    random_state=RANDOM_STATE
)

final_model.fit(X_full, y_full)


In [ ]:
# Predict test set and export submission CSV

test_pred = final_model.predict(X_test)
test_pred = np.clip(test_pred, 0, 20)

# Match predictions back to Kaggle test IDs
test_ids = matrix.loc[test_mask, :].reset_index(drop=True)
test_ids = test_ids.merge(test[["ID", "shop_id", "item_id"]], on=["shop_id", "item_id"], how="left")

submission = pd.DataFrame({
    "ID": test_ids["ID"].astype(int),
    "item_cnt_month": test_pred
}).sort_values("ID")

submission.to_csv("submissionfile.csv", index=False)
submission.head()
